In [113]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
df = pd.read_csv("/content/pima-indians-diabetes.csv")
df.head()


,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [114]:
df.columns =['F1', 'F2', 'F3', 'F4', 'F5','F6','F7','F8',   'Y']
df

,F1,F2,F3,F4,F5,F6,F7,F8,Y
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0
...,...,...,...,...,...,...,...,...,...
762,10,101,76,48,180,32.9,0.171,63,0
763,2,122,70,27,0,36.8,0.340,27,0
764,5,121,72,23,112,26.2,0.245,30,0
765,1,126,60,0,0,30.1,0.349,47,1


In [115]:
zero_columns = [col for col in df.columns if (df[col].min() == 0 and col not in ["Y", "F1"])]
for col in zero_columns:
    df[col] = np.where(df[col] == 0, np.nan, df[col])

df.isna().sum()

F1      0
F2      5
F3     35
F4    227
F5    373
F6     11
F7      0
F8      0
Y       0
dtype: int64

In [116]:
for col in zero_columns:
    df.loc[df[col].isnull(), col] = df[col].mean()

df.isna().sum()

F1    0
F2    0
F3    0
F4    0
F5    0
F6    0
F7    0
F8    0
Y     0
dtype: int64

In [117]:
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values

In [118]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y , test_size = 0.25, random_state = 0)

In [119]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.transform(X_test)


# Models
## KNN

In [120]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score ,precision_score ,f1_score
from time import process_time

model_list = []
model_test_acc = []
model_recall = []
model_precision = []
model_F1 = []
model_time = []

for i in range(1,10,2):
  start = process_time()

  Knn = KNeighborsClassifier(n_neighbors = i)
  Knn.fit(X_train, y_train)
  Knn_pred = Knn.predict(X_test)

  end = process_time()

  acc= accuracy_score(y_true = y_test, y_pred=Knn_pred)
  recall= recall_score(y_true = y_test, y_pred=Knn_pred)
  t = end-start
  per = precision_score(y_true = y_test, y_pred=Knn_pred)
  f1 = f1_score(y_true = y_test, y_pred=Knn_pred)

  model_list.append(str(i) +" neighbors")
  model_test_acc.append(acc)
  model_recall.append(recall)
  model_precision.append(per)
  model_F1.append(f1)
  model_time.append(t)

  print("n_neighbors = ",i)
  print("test acc:",acc)
  print("Recall :",recall)
  print("Recall :",per)
  print("Recall :",f1)

  print("Time :",t,"\n")




n_neighbors =  1
test acc: 0.703125
Recall : 0.5081967213114754
Recall : 0.5344827586206896
Recall : 0.5210084033613445
Time : 0.014876896999993505 

n_neighbors =  3
test acc: 0.7291666666666666
Recall : 0.5573770491803278
Recall : 0.576271186440678
Recall : 0.5666666666666667
Time : 0.016972180999999864 

n_neighbors =  5
test acc: 0.734375
Recall : 0.5573770491803278
Recall : 0.5862068965517241
Recall : 0.5714285714285715
Time : 0.015406794000000446 

n_neighbors =  7
test acc: 0.7760416666666666
Recall : 0.6065573770491803
Recall : 0.6607142857142857
Recall : 0.6324786324786325
Time : 0.015837508000004163 

n_neighbors =  9
test acc: 0.75
Recall : 0.4918032786885246
Recall : 0.6382978723404256
Recall : 0.5555555555555556
Time : 0.01629388199998516 



## SVM

In [121]:
from sklearn.svm import SVC

model_train_acc = []


SVM_list = ['linear', 'poly', 'rbf', 'sigmoid']
for model in SVM_list:
  start = process_time()
  svm_model = SVC(kernel = model, random_state = i)
  svm_model.fit(X_train, y_train)

  model_pred = svm_model.predict(X_test)
  end = process_time()


  print(model," train acc:",accuracy_score(y_true = y_train, y_pred=svm_model.predict(X_train)))
  print(model," test acc:",accuracy_score(y_true = y_test, y_pred=model_pred))
  print(model,"Recall :",(recall_score(y_true = y_test, y_pred=model_pred)))
  print(model,"precision :",(precision_score(y_true = y_test, y_pred=model_pred)))
  print(model,"F1 :",(f1_score(y_true = y_test, y_pred=model_pred,)))
  print(model,"Time :",str(end-start),"\n")

  model_list.append("SVM_"+model)
  model_test_acc.append(accuracy_score(y_true = y_test, y_pred=model_pred))
  model_train_acc.append(accuracy_score(y_true = y_train, y_pred=svm_model.predict(X_train)))
  model_recall.append(recall_score(y_true = y_test, y_pred=model_pred,))
  model_precision.append(precision_score(y_true = y_test, y_pred=model_pred))
  model_F1.append(f1_score(y_true = y_test, y_pred=model_pred))
  model_time.append(str((end-start)))


linear  train acc: 0.76
linear  test acc: 0.8020833333333334
linear Recall : 0.5409836065573771
linear precision : 0.7674418604651163
linear F1 : 0.6346153846153847
linear Time : 0.017394353999975465 

poly  train acc: 0.8052173913043478
poly  test acc: 0.7708333333333334
poly Recall : 0.5573770491803278
poly precision : 0.6666666666666666
poly F1 : 0.6071428571428571
poly Time : 0.04456011800002102 

rbf  train acc: 0.7965217391304348
rbf  test acc: 0.78125
rbf Recall : 0.5737704918032787
rbf precision : 0.6862745098039216
rbf F1 : 0.625
rbf Time : 0.02321598099999278 

sigmoid  train acc: 0.35478260869565215
sigmoid  test acc: 0.3645833333333333
sigmoid Recall : 0.09836065573770492
sigmoid precision : 0.0821917808219178
sigmoid F1 : 0.08955223880597014
sigmoid Time : 0.03198925099999883 



## Decision Tree

In [122]:
from sklearn import tree
start = process_time()

tree = tree.DecisionTreeClassifier(random_state=218)
tree.fit(X_train,y_train)
treepred = tree.predict(X_test)
end = process_time()

print("Train acc:",accuracy_score(y_true = y_train, y_pred=tree.predict(X_train)))
print("Test acc:",accuracy_score(y_true = y_test, y_pred=tree_pred))
print("Recall :",(recall_score(y_true = y_test, y_pred=tree_pred)))
print("precision :",(precision_score(y_true = y_test, y_pred=tree_pred,)))
print("F1 :",(f1_score(y_true = y_test, y_pred=tree_pred)))
print("Time :",str(end-start),"\n")

model_list.append("DecesionTree")
model_test_acc.append(accuracy_score(y_true = y_test, y_pred=tree_pred))
model_train_acc.append(accuracy_score(y_true = y_train, y_pred=tree.predict(X_train)))
model_recall.append(recall_score(y_true = y_test, y_pred=tree_pred,))
model_precision.append(precision_score(y_true = y_test, y_pred=tree_pred,))
model_F1.append(f1_score(y_true = y_test, y_pred=tree_pred,))

model_time.append(str((end-start)))

Train acc: 1.0
Test acc: 0.7135416666666666
Recall : 0.5573770491803278
precision : 0.5483870967741935
F1 : 0.5528455284552846
Time : 0.00597858099999371 



## Random Forest

In [123]:
from sklearn.ensemble import RandomForestClassifier
start = process_time()

RF = RandomForestClassifier(n_estimators = 5,  max_depth=5, random_state = 288)
RF.fit(X_train, y_train)
RF_pred = RF.predict(X_test)
end = process_time()


print("RF train acc:",accuracy_score(y_true = y_train, y_pred=RF.predict(X_train)))
print("RF test acc:",accuracy_score(y_true = y_test, y_pred=RF_pred))
print("Recall :",(recall_score(y_true = y_test, y_pred=RF_pred,)))
print("precision :",(precision_score(y_true = y_test, y_pred=RF_pred,)))
print("F1 :",(f1_score(y_true = y_test, y_pred=RF_pred,)))
print("Time :",str(end-start),"\n")


model_list.append("Random_forst")
model_test_acc.append(accuracy_score(y_true = y_train, y_pred=RF.predict(X_train)))
model_train_acc.append(accuracy_score(y_true = y_test, y_pred=RF_pred))
model_recall.append(recall_score(y_true = y_test, y_pred=RF_pred,))
model_precision.append(precision_score(y_true = y_test, y_pred=RF_pred,))
model_F1.append(f1_score(y_true = y_test, y_pred=RF_pred,))
model_time.append(str((end-start)))


RF train acc: 0.8191304347826087
RF test acc: 0.8385416666666666
Recall : 0.6557377049180327
precision : 0.8
F1 : 0.7207207207207207
Time : 0.019710951000007526 



## Boost Classifier

In [124]:
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
start = process_time()

AdaBoost = AdaBoostClassifier(n_estimators=5, random_state=0)
AdaBoost.fit(X_train, y_train)
Boost_pred = AdaBoost.predict(X_test)

end = process_time()



print("boost train acc:",accuracy_score(y_true = y_train, y_pred=AdaBoost.predict(X_train)))
print("boost test acc:",accuracy_score(y_true = y_test, y_pred=Boost_pred))
print("Recall :",(recall_score(y_true = y_test, y_pred=Boost_pred,)))
print("precision :",(precision_score(y_true = y_test, y_pred=Boost_pred,)))
print("F1 :",(f1_score(y_true = y_test, y_pred=Boost_pred,)))
print("Time :",str(end-start),"\n")

model_list.append("Boost")
model_test_acc.append(accuracy_score(y_true = y_train, y_pred=AdaBoost.predict(X_train)))
model_train_acc.append(accuracy_score(y_true = y_test, y_pred=Boost_pred))
model_recall.append(recall_score(y_true = y_test, y_pred=Boost_pred,))
model_precision.append(precision_score(y_true = y_test, y_pred=Boost_pred))
model_F1.append(f1_score(y_true = y_test, y_pred=Boost_pred,))
model_time.append(str((end-start)))

boost train acc: 0.7634782608695653
boost test acc: 0.7708333333333334
Recall : 0.5573770491803278
precision : 0.6666666666666666
F1 : 0.6071428571428571
Time : 0.020275955000016666 



## Logistic Regression

In [125]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(random_state=0)
LR.fit(X_train, y_train)

LR_pred = LR.predict(X_test)

print("RF train acc:",accuracy_score(y_true = y_train, y_pred=LR.predict(X_train)))
print("RF test acc:",accuracy_score(y_true = y_test, y_pred=LR_pred))
print("Recall :",(recall_score(y_true = y_test, y_pred=LR_pred,)))
print("precision :",(precision_score(y_true = y_test, y_pred=LR_pred,)),)
print("F1 :",(f1_score(y_true = y_test, y_pred=LR_pred,)),)

print("Time :",str(end-start),"\n")

model_list.append("LogisticRegression")
model_test_acc.append(accuracy_score(y_true = y_train, y_pred=LR.predict(X_train)))
model_train_acc.append(accuracy_score(y_true = y_test, y_pred=LR_pred))
model_recall.append(recall_score(y_true = y_test, y_pred=LR_pred, ))
model_precision.append(precision_score(y_true = y_test, y_pred=LR_pred, ))
model_F1.append(f1_score(y_true = y_test, y_pred=LR_pred,))
model_time.append(str((end-start)))

RF train acc: 0.7617391304347826
RF test acc: 0.78125
Recall : 0.5245901639344263
precision : 0.7111111111111111
F1 : 0.6037735849056605
Time : 0.020275955000016666 



## Naive Bayes

In [126]:
from sklearn.naive_bayes import GaussianNB , MultinomialNB
from time import process_time
from sklearn.metrics import accuracy_score, recall_score ,precision_score ,f1_score

start = process_time()
classifier = GaussianNB()
classifier.fit(X_train, y_train)
GauNB_pred = classifier.predict(X_test)
end = process_time()

model_list.append("GaussianNB")
model_test_acc.append(accuracy_score(y_true = y_test, y_pred=GauNB_pred))
model_train_acc.append(accuracy_score(y_true = y_train, y_pred=classifier.predict(X_train)))
model_recall.append(recall_score(y_true = y_test, y_pred=GauNB_pred,))
model_precision.append(precision_score(y_true = y_test, y_pred=GauNB_pred,))
model_F1.append(f1_score(y_true = y_test, y_pred=GauNB_pred))
model_time.append(str((end-start)))

print("GaussianNB train acc:",accuracy_score(y_true = y_train, y_pred=classifier.predict(X_train)))
print("GaussianNB test acc:",accuracy_score(y_true = y_test, y_pred=GauNB_pred))
print("GaussianNB Recall :",(recall_score(y_true = y_test, y_pred=GauNB_pred,)))
print("GaussianNB precision :",(precision_score(y_true = y_test, y_pred=GauNB_pred)))
print("GaussianNB F1 :",(f1_score(y_true = y_test, y_pred=GauNB_pred)))
print("GaussianNB Time :",str(end-start),"\n")


start = process_time()
classifier1 = MultinomialNB()
classifier1.fit(X_train, y_train)
MulNB_pred = classifier1.predict(X_test)
end = process_time()

model_list.append("MultinomialNB")
model_test_acc.append(accuracy_score(y_true = y_test, y_pred=MulNB_pred))
model_train_acc.append(accuracy_score(y_true = y_train, y_pred=classifier1.predict(X_train)))
model_recall.append(recall_score(y_true = y_test, y_pred=MulNB_pred))
model_precision.append(precision_score(y_true = y_test, y_pred=MulNB_pred))
model_F1.append(f1_score(y_true = y_test, y_pred=MulNB_pred))
model_time.append(str((end-start)))

print("MultinomialNB train acc:",accuracy_score(y_true = y_train, y_pred=classifier1.predict(X_train)))
print("MultinomialNB test acc:",accuracy_score(y_true = y_test, y_pred=MulNB_pred))
print("MultinomialNB Recall :",(recall_score(y_true = y_test, y_pred=MulNB_pred,)))
print("MultinomialNB precision :",(precision_score(y_true = y_test, y_pred=MulNB_pred)))
print("MultinomialNB F1 :",(f1_score(y_true = y_test, y_pred=MulNB_pred)))
print("MultinomialNB Time :",str(end-start),"\n")


GaussianNB train acc: 0.7426086956521739
GaussianNB test acc: 0.7760416666666666
GaussianNB Recall : 0.6065573770491803
GaussianNB precision : 0.6607142857142857
GaussianNB F1 : 0.6324786324786325
GaussianNB Time : 0.002758466000017279 

MultinomialNB train acc: 0.6417391304347826
MultinomialNB test acc: 0.6822916666666666
MultinomialNB Recall : 0.0
MultinomialNB precision : 0.0
MultinomialNB F1 : 0.0
MultinomialNB Time : 0.002748077000006788 



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Tables

In [127]:
df = pd.DataFrame(model_list, columns=['number of neighbors'])
df['test acc'] = model_test_acc
df['Recall '] = model_recall
df['prec_score'] = model_precision
df['F1_score'] = model_F1
df['Time '] = model_time


print("sorting by acc")
df.sort_values(by=['test acc'], ascending = False)

sorting by acc


,number of neighbors,test acc,Recall,prec_score,F1_score,Time
10,Random_forst,0.819130,0.655738,0.800000,0.720721,0.019710951000007526
5,SVM_linear,0.802083,0.540984,0.767442,0.634615,0.017394353999975465
7,SVM_rbf,0.781250,0.573770,0.686275,0.625000,0.02321598099999278
3,7 neighbors,0.776042,0.606557,0.660714,0.632479,0.015838
13,GaussianNB,0.776042,0.606557,0.660714,0.632479,0.002758466000017279
6,SVM_poly,0.770833,0.557377,0.666667,0.607143,0.04456011800002102
11,Boost,0.763478,0.557377,0.666667,0.607143,0.020275955000016666
12,LogisticRegression,0.761739,0.524590,0.711111,0.603774,0.020275955000016666
4,9 neighbors,0.750000,0.491803,0.638298,0.555556,0.016294
2,5 neighbors,0.734375,0.557377,0.586207,0.571429,0.015407


In [128]:
print("sorting by rcall")
df.sort_values(by=['Recall '], ascending = False)

sorting by rcall


,number of neighbors,test acc,Recall,prec_score,F1_score,Time
10,Random_forst,0.819130,0.655738,0.800000,0.720721,0.019710951000007526
3,7 neighbors,0.776042,0.606557,0.660714,0.632479,0.015838
13,GaussianNB,0.776042,0.606557,0.660714,0.632479,0.002758466000017279
7,SVM_rbf,0.781250,0.573770,0.686275,0.625000,0.02321598099999278
1,3 neighbors,0.729167,0.557377,0.576271,0.566667,0.016972
2,5 neighbors,0.734375,0.557377,0.586207,0.571429,0.015407
6,SVM_poly,0.770833,0.557377,0.666667,0.607143,0.04456011800002102
9,DecesionTree,0.713542,0.557377,0.548387,0.552846,0.00597858099999371
11,Boost,0.763478,0.557377,0.666667,0.607143,0.020275955000016666
5,SVM_linear,0.802083,0.540984,0.767442,0.634615,0.017394353999975465


# Random has best recall and best accuracy but not best enough for medical perposes